In [0]:
%sql
-- Create Customers Table
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    signup_date DATE
);

-- Create Orders Table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4610649436339893>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', '-- Create Customers Table\nCREATE TABLE customers (\n    customer_id INT PRIMARY KEY,\n    name VARCHAR(100),\n    email VARCHAR(100),\n    signup_date DATE\n);\n\n-- Create Orders Table\nCREATE TABLE orders (\n    order_id INT PRIMARY KEY,\n    customer_id INT,\n    order_date DATE,\n    amount DECIMAL(10,2),\n    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)\n);\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decor

In [0]:
%sql
-- Insert Customers
INSERT INTO customers (customer_id, name, email, signup_date) VALUES
(1, 'Alice Johnson', 'alice@example.com', '2020-01-15'),
(2, 'Bob Smith', 'bob@example.com', '2019-06-10'),
(3, 'Charlie Lee', 'charlie@example.com', '2021-03-22'),
(4, 'Diana Evans', 'diana@example.com', '2020-11-01');

-- Insert Orders
INSERT INTO orders (order_id, customer_id, order_date, amount) VALUES
(101, 1, '2020-02-01', 1200.00),
(102, 1, '2020-05-15', 800.00),
(103, 2, '2019-07-10', 3000.00),
(104, 2, '2019-12-20', 2500.00),
(105, 3, '2021-04-01', 600.00),
(106, 3, '2021-07-19', 700.00),
(107, 4, '2020-12-05', 1500.00),
(108, 4, '2021-01-10', 2000.00);


num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
-- Remove duplicate orders
DELETE FROM orders
WHERE order_id IN (
    SELECT order_id
    FROM (
        SELECT order_id,
               ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY order_date) AS rn
        FROM orders
    ) t
    WHERE rn > 1
);

-- Replace NULL amounts with 0
UPDATE orders
SET amount = 0
WHERE amount IS NULL;


num_affected_rows
0


In [0]:
%sql
WITH customer_spending AS (
    SELECT 
        c.customer_id,
        c.name,
        SUM(o.amount) AS total_spent,
        COUNT(o.order_id) AS total_orders,
        AVG(o.amount) AS avg_order_value
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name
)
SELECT * FROM customer_spending;


customer_id,name,total_spent,total_orders,avg_order_value
1,Alice Johnson,2000.00,2,1000.000000
4,Diana Evans,3500.00,2,1750.000000
3,Charlie Lee,1300.00,2,650.000000
2,Bob Smith,5500.00,2,2750.000000


In [0]:
%sql
WITH customer_spending AS (
    SELECT 
        c.customer_id,
        c.name,
        SUM(o.amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name
)
SELECT 
    customer_id,
    name,
    total_spent,
    CASE
        WHEN total_spent >= 5000 THEN 'Platinum'
        WHEN total_spent BETWEEN 2000 AND 4999 THEN 'Gold'
        ELSE 'Silver'
    END AS customer_segment
FROM customer_spending;


customer_id,name,total_spent,customer_segment
1,Alice Johnson,2000.00,Gold
4,Diana Evans,3500.00,Gold
3,Charlie Lee,1300.00,Silver
2,Bob Smith,5500.00,Platinum


In [0]:
%sql
CREATE TABLE customer_segments AS
WITH customer_spending AS (
    SELECT 
        c.customer_id,
        c.name,
        SUM(o.amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.name
)
SELECT 
    customer_id,
    name,
    total_spent,
    CASE
        WHEN total_spent >= 5000 THEN 'Platinum'
        WHEN total_spent BETWEEN 2000 AND 4999 THEN 'Gold'
        ELSE 'Silver'
    END AS customer_segment
FROM customer_spending;


num_affected_rows,num_inserted_rows
